In [ ]:
import requests
import os

NOTION_API_KEY = os.environ['NOTION_API_KEY']
NOTION_DATABASE_ID = os.environ['NOTION_DATABASE_ID']
NOTION_TEMPLATE_PAGE_ID = os.environ['NOTION_TEMPLATE_PAGE_ID']

HEADERS = {
    "Notion-Version": "2022-06-28",
    "Authorization": f"Bearer {NOTION_API_KEY}",
    "Content-Type": "application/json"
}

def get_blocks(page_id):
    url = f"https://api.notion.com/v1/blocks/{page_id}/children"
    res = requests.get(url, headers=HEADERS)
    blocks = res.json().get("results", [])
    # Notionのchildren階層は1段のみ。必要なら再帰で全階層取得
    return blocks

def filter_block(block):
    # API経由append用に余計なフィールド削除
    return {
        "object": "block",
        "type": block["type"],
        block["type"]: block[block["type"]]
    }

def create_page():
    url = "https://api.notion.com/v1/pages"
    data = {
        "parent": { "database_id": NOTION_DATABASE_ID },
        "properties": {
            "名前": {
                "title": [{"text": {"content": "テンプレ複製テスト"}}]
            }
            
        }
    }
    res = requests.post(url, headers=HEADERS, json=data)
    json_res = res.json()
    return json_res["id"]  # 新ページID

def append_blocks(page_id, blocks):
    url = f"https://api.notion.com/v1/blocks/{page_id}/children"
    data = { "children": [filter_block(b) for b in blocks] }
    res = requests.patch(url, headers=HEADERS, json=data)
    return res.json()


template_blocks = get_blocks(NOTION_TEMPLATE_PAGE_ID)
new_page_id = create_page()
append_blocks(new_page_id, template_blocks)

{'object': 'error',
 'status': 400,
 'code': 'validation_error',
 'message': 'body failed validation: body.children[2].callout.icon should be an object or `undefined`, instead was `null`.',
 'request_id': 'f3c88aae-50b0-4b3a-b667-82258edca055'}